## [Start Milvus](https://milvus.io/docs/install_standalone-docker.md#Start-Milvus)

```bash
sudo docker compose run -d
```

- Check everything runs correctly

In [1]:
!docker compose ps


NAME                IMAGE                                      COMMAND                  SERVICE             CREATED             STATUS                            PORTS
milvus-etcd         quay.io/coreos/etcd:v3.5.5                 "etcd -advertise-cli…"   etcd                6 seconds ago       Up 5 seconds (health: starting)   2379-2380/tcp
milvus-minio        minio/minio:RELEASE.2023-03-20T20-16-18Z   "/usr/bin/docker-ent…"   minio               6 seconds ago       Up 4 seconds (health: starting)   0.0.0.0:9000-9001->9000-9001/tcp, :::9000-9001->9000-9001/tcp
milvus-standalone   milvusdb/milvus:v2.3.3                     "/tini -- milvus run…"   standalone          6 seconds ago       Up 4 seconds (health: starting)   0.0.0.0:9091->9091/tcp, :::9091->9091/tcp, 0.0.0.0:19530->19530/tcp, :::19530->19530/tcp


- Get the port of Milvus:

In [2]:
!docker port milvus-standalone 19530/tcp


0.0.0.0:19530
[::]:19530


## Example Run

- Imports a PyMilvus package:


In [3]:
from pymilvus import (
    connections,
    utility,
    FieldSchema,
    CollectionSchema,
    DataType,
    Collection,
)

- Connects to a server:


In [4]:
connections.connect("default", host="localhost", port="19530")


- Creates a collection:
 

In [5]:
fields = [
    FieldSchema(name="pk", dtype=DataType.INT64, is_primary=True, auto_id=False),
    FieldSchema(name="random", dtype=DataType.DOUBLE),
    FieldSchema(name="embeddings", dtype=DataType.FLOAT_VECTOR, dim=8)
]
schema = CollectionSchema(fields, "hello_milvus is the simplest demo to introduce the APIs")
hello_milvus = Collection("hello_milvus", schema)
hello_milvus

<Collection>:
-------------
<name>: hello_milvus
<description>: hello_milvus is the simplest demo to introduce the APIs
<schema>: {'auto_id': False, 'description': 'hello_milvus is the simplest demo to introduce the APIs', 'fields': [{'name': 'pk', 'description': '', 'type': <DataType.INT64: 5>, 'is_primary': True, 'auto_id': False}, {'name': 'random', 'description': '', 'type': <DataType.DOUBLE: 11>}, {'name': 'embeddings', 'description': '', 'type': <DataType.FLOAT_VECTOR: 101>, 'params': {'dim': 8}}]}

- Inserts vectors in the collection:


In [6]:
import random
entities = [
    [i for i in range(3000)],  # field pk
    [float(random.randrange(-20, -10)) for _ in range(3000)],  # field random
    [[random.random() for _ in range(8)] for _ in range(3000)],  # field embeddings
]
insert_result = hello_milvus.insert(entities)
hello_milvus.flush()  


- Builds indexes on the entities:


In [7]:
index = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
hello_milvus.create_index("embeddings", index)


Status(code=0, message=)

- Loads the collection to memory and performs a vector similarity search:


In [8]:
hello_milvus.load()
vectors_to_search = entities[-1][-2:]
search_params = {
    "metric_type": "L2",
    "params": {"nprobe": 10},
}
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, output_fields=["random"]) # Similarity search
result

['["id: 2998, distance: 0.0, entity: {\'random\': -15.0}", "id: 2347, distance: 0.1160244420170784, entity: {\'random\': -15.0}", "id: 302, distance: 0.20034097135066986, entity: {\'random\': -12.0}"]', '["id: 2999, distance: 0.0, entity: {\'random\': -12.0}", "id: 164, distance: 0.12360914051532745, entity: {\'random\': -20.0}", "id: 724, distance: 0.14829647541046143, entity: {\'random\': -14.0}"]']

- Performs a vector query:


In [9]:
result = hello_milvus.query(expr="random > -14", output_fields=["random", "embeddings"]) # Query based on expr
result[:10]

[{'embeddings': [0.46464592,
   0.19173644,
   0.44714734,
   0.23775755,
   0.36889437,
   0.23809732,
   0.39155307,
   0.9410035],
  'pk': 6,
  'random': -11.0},
 {'embeddings': [0.34762254,
   0.2946214,
   0.24401015,
   0.163827,
   0.6607513,
   0.565676,
   0.90860325,
   0.47596365],
  'pk': 10,
  'random': -13.0},
 {'embeddings': [0.18230979,
   0.55295897,
   0.92754215,
   0.16744952,
   0.60794026,
   0.06433548,
   0.9459016,
   0.2037491],
  'pk': 14,
  'random': -11.0},
 {'embeddings': [0.4370434,
   0.13192105,
   0.6272539,
   0.0057994244,
   0.2813594,
   0.5032944,
   0.96526426,
   0.97722006],
  'pk': 19,
  'random': -12.0},
 {'embeddings': [0.6337357,
   0.7868831,
   0.5248237,
   0.12394486,
   0.93866414,
   0.21057846,
   0.5234568,
   0.3306302],
  'pk': 20,
  'random': -12.0},
 {'embeddings': [0.28544936,
   0.6728877,
   0.7315763,
   0.79570043,
   0.65235496,
   0.5261155,
   0.61333984,
   0.7120516],
  'pk': 24,
  'random': -12.0},
 {'embeddings': [0.

- Performs a hybrid search:


In [10]:
result = hello_milvus.search(vectors_to_search, "embeddings", search_params, limit=3, expr="random > -12", output_fields=["random"])
result

['["id: 2293, distance: 0.5049536824226379, entity: {\'random\': -11.0}", "id: 1061, distance: 0.5454369783401489, entity: {\'random\': -11.0}", "id: 2808, distance: 0.5913836359977722, entity: {\'random\': -11.0}"]', '["id: 2738, distance: 0.20923709869384766, entity: {\'random\': -11.0}", "id: 1793, distance: 0.25962626934051514, entity: {\'random\': -11.0}", "id: 958, distance: 0.3039100766181946, entity: {\'random\': -11.0}"]']

- Deletes entities by their primary keys:


In [11]:
expr = f"pk in [{entities[0][0]}, {entities[0][1]}]"
hello_milvus.delete(expr)


(insert count: 0, delete count: 2, upsert count: 0, timestamp: 445611827024625665, success count: 0, err count: 0)

- Drops the collection:


In [12]:
utility.drop_collection("hello_milvus")


Congratulations! You have started Milvus standalone and performed your first vector similarity search.

## [Stop Milvus](https://milvus.io/docs/install_standalone-docker.md#Stop-Milvus)

In [13]:
!docker compose down

[+] Running 0/0
 ⠋ Container milvus-standalone  Stopping                                   0.1s 
[+] Running 0/1
 ⠙ Container milvus-standalone  Stopping                                   0.2s 
[+] Running 0/1
 ⠹ Container milvus-standalone  Stopping                                   0.3s 
[+] Running 0/1
 ⠸ Container milvus-standalone  Stopping                                   0.4s 
[+] Running 0/1
 ⠼ Container milvus-standalone  Stopping                                   0.5s 
[+] Running 0/1
 ⠴ Container milvus-standalone  Stopping                                   0.6s 
[+] Running 0/1
 ⠦ Container milvus-standalone  Stopping                                   0.7s 
[+] Running 0/1
 ⠧ Container milvus-standalone  Stopping                                   0.8s 
[+] Running 0/1
 ⠇ Container milvus-standalone  Stopping                                   0.9s 
[+] Running 0/1
 ⠏ Container milvus-standalone  Stopping                                   1.0s 
[+] Running 0/1
 ⠋ Container m

- To delete the data, delete the `volume` directory.